In [10]:
import pandas as pd
from river import stream,tree,metrics
import utils
from encoding import prefix_bin
import numpy as np
from sklearn.metrics import classification_report, accuracy_score
import os
from tqdm import tqdm
import sliding_window
import xgboost as xgb

import datetime, time
import importlib
importlib.reload(sliding_window)

<module 'sliding_window' from 'C:\\Users\\suhwanlee\\Desktop\\project\\streaming_anomaly_detect\\sliding_window.py'>

In [11]:
file_name = './data/loan_baseline.pnml_noise_0.15_iteration_1_seed_614_sample.csv'


dataset = stream.iter_csv(
            file_name
#             './data/loan_baseline.pnml_noise_0.15_iteration_1_seed_614_simple.csv',
            )

totallength = len(list(dataset))

In [12]:
dataset = stream.iter_csv(
            file_name,
            drop=['noise', 'lifecycle:transition', 'Variant', 'Variant index'],
            )
enctype = 'Index-base'

In [13]:
key_pair = {
'Case ID':'caseid',
'Activity':'activity',
# 'Resource':'resource',
'Complete Timestamp':'ts',
}
catatars= ['activity']#,'resource']

case_dict ={}
training_models ={}

casecount = 0
rowcounter = 0
resultdict ={}
acc_dict ={}
prefix_wise_window = {}
prediction_result = {}
graceperiod_finish=0
finishedcases = set()

In [14]:
# Sliding window for training setting
window_size = 100
retraining_size = 20
training_window = sliding_window.training_window(window_size,retraining_size)

In [15]:
def display_progress(row_counting, total_length, interval=500):
    if rowcounter%interval == 0:
        print(round(rowcounter*100/totallength,2) ,'%', 'Case finished: %s'%(casecount), 'Running cases: %s'%(len(case_dict)))

In [16]:
def training_stage(window, training_models):
    '''
    Manage training stage of streaming anomaly detection
    ----------
    Parameters
    window: class training_window
        Sliding window with training data
    training_models: dict
        Trained detector by prefix stored in. Default is randomforest
    ----------
    Return
    training_models
    '''
    pw_window = window.prefix_wise_window()
    for x in pw_window:
        clf  = xgb.XGBClassifier(n_estimators = 100, learning_rate=0.01)
        training_x = pw_window[x][0]
        training_y = pw_window[x][1]
        
        clf.fit(pw_window[x][0],pw_window[x][1])
        if 'detector_%s'%(x) not in training_models:
            training_models['detector_%s'%(x)] =[0,0]
        training_models['detector_%s'%(x)][0] += 1
        training_models['detector_%s'%(x)][1] = clf
    return training_models

In [17]:
def predict_activity_proba(last_event):
    '''
    Predict next activity prediction 
    
    Parameters
    ----------
    last_event: case_bin
    
    Return
    ----------
    modelid, prediction
    
    '''
    feature_matrix = prefix_wise_window['window_%s'%(last_event.prefix_length)][0].columns.values
    current_event = utils.readjustment_training(last_event.encoded, feature_matrix)
    current_event = pd.Series(current_event).to_frame().T
    prediction = [training_models['detector_window_%s'%(last_event.prefix_length)][1].predict_proba(current_event), training_models['detector_window_%s'%(last_event.prefix_length)][1].classes_]
    modelid = training_models['detector_window_%s'%(last_event.prefix_length)][0]

    return modelid, prediction

In [18]:
def first_event(case_bin):
    '''
    Generate start event before first event
    '''
    print(case_bin.event['ts'])
    empty_data ={'activity':'Start signal', 'ts':datetime.datetime.strftime(case_bin.event['ts'], '%Y-%m-%d %H:%M:%S')}
    start_event = prefix_bin(case_bin.caseid, empty_data)
    start_event.set_prefix_length(0)
    start_event.update_encoded(catattrs=catatars,enctype=enctype)
    start_event.update_truelabel(case_bin.event['activity'])
    return start_event

In [19]:
training_time = []

In [20]:
start_time = time.time()

for x,y in dataset:
    display_progress(rowcounter, totallength)
    rowcounter +=1
    
    utils.dictkey_chg(x, key_pair)
    # Event stream change dictionary keys
    x['ts'] = x['ts'][:-4]
    
    # Check label possible
    
    # Initialize case by prefix length
    caseid = x['caseid']
    x.pop('caseid')
    
    case_bin = prefix_bin(caseid, x)
    
    if caseid not in list(case_dict.keys()):
        case_dict[caseid] = []
        case_bin.set_prefix_length(1)
        
    elif caseid in finishedcases:
        continue
    
    else:
        case_bin.set_prefix_length(len(case_dict[caseid])+1)
        case_bin.set_prev_enc(case_dict[caseid][-1])
    
    # Encode event and cases and add to DB
    ts = case_bin.event['ts']
    case_bin.update_encoded(catattrs=catatars,enctype=enctype)
    
    # Set current activity as outcome of previous event
    if case_bin.prefix_length != 1:
        case_bin.prev_enc.update_truelabel(x['activity'])

    # First prediction for current event
    
    last_event = case_bin
    modelid = 'None'
    prediction = 'Not Available'

    if len(training_window.getAllitems()) !=0:
        if 'window_%s'%(last_event.prefix_length) in list(prefix_wise_window.keys()) and 'detector_window_%s'%(last_event.prefix_length) in training_models.keys():
            modelid, prediction = predict_activity_proba(last_event)
    case_bin.update_prediction((modelid, (prediction,ts)))        
            
    # Update training window and finish the case
    if x['activity'] == 'End':
        training_window.update_window({caseid: case_dict[caseid]})        
        if training_window.retraining == training_window.retraining_count:            
            train_start = time.time()
            training_models = training_stage(training_window, training_models)
            train_end = time.time()
            training_time.append(train_end-train_start)

            prefix_wise_window = training_window.prefix_wise_window()
            
        resultdict[caseid] = case_dict[caseid]
        case_dict.pop(caseid)

        casecount +=1
        for x in case_dict:
            last_event = case_dict[x][-1]
            modelid = 'None'
            prediction = 'Not Available'

            if len(training_window.getAllitems()) !=0:
                prefix_wise_window = training_window.prefix_wise_window()
                if 'window_%s'%(last_event.prefix_length) in list(prefix_wise_window.keys()) and 'detector_window_%s'%(last_event.prefix_length) in training_models.keys():
                    modelid, prediction = predict_activity_proba(last_event)

#                     feature_matrix = prefix_wise_window['window_%s'%(last_event.prefix_length)][0].columns.values
#                     current_event = utils.readjustment_training(last_event.encoded, feature_matrix)
#                     current_event = pd.Series(current_event).to_frame().T
#                     prediction = [training_models['detector_window_%s'%(last_event.prefix_length)][1].predict_proba(current_event), training_models['detector_window_%s'%(last_event.prefix_length)][1].classes_]
#                     modelid = training_models['detector_window_%s'%(last_event.prefix_length)][0]
            case_dict[x][-1].update_prediction((modelid, (prediction,ts)))        
        training_window.reset_retraining_count()
    else:
        case_dict[caseid].append(case_bin)


end_time = time.time()

0.0 % Case finished: 0 Running cases: 0
[12:49:27] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


c:\python\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[12:49:28] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:49:28] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:49:28] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:49:28] WARNING: C:/Users/Administrator/workspace/xgboost-win64_

[12:49:31] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:49:31] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:49:31] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
5.8 % Case finished: 29 Running cases: 1
[12:49:32] WARNING: C:/Users/

[12:49:36] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:49:36] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:49:36] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:49:36] WARNING: C:/Users/Administrator/workspace/xgboost-win64_

...

[12:52:16] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:52:16] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:52:16] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:52:16] WARNING: C:/Users/Administrator/workspace/xgboost-win64_

[12:52:22] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:52:22] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:52:22] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:52:23] WARNING: C:/Users/Administrator/workspace/xgboost-win64_

[12:52:27] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:52:28] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:52:28] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:52:28] WARNING: C:/Users/Administrator/workspace/xgboost-win64_

[12:52:34] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:52:34] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:52:35] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:52:35] WARNING: C:/Users/Administrator/workspace/xgboost-win64_

[12:52:41] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:52:41] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:52:42] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:52:42] WARNING: C:/Users/Administrator/workspace/xgboost-win64_

In [63]:
print((end_time-start_time)/60)

4.259771056969961


In [65]:
original_df = pd.read_csv(file_name)

In [66]:
for_confusion_matrix = {}

counting_normal = 0

for threshold in [0.01,0.05,0.1,0.15,0.2,0.25]:
    global_true =[]
    global_pred = []

    for caseid in list(resultdict.keys()):

        for_confusion_matrix[int(caseid)] =[]

        prediction_list = []

        df = original_df[original_df['Case ID'] == int(caseid)].reset_index(drop=True)
        for pos, t in enumerate(resultdict['%s'%(caseid)]):
            prediction_label = 'Normal'

            predictions = list(t.predicted.values())[0][0]
            predictions_proba = predictions[0][0]
            predictions_value = list(predictions[1])
            if predictions  == 'Not Available':
                prediction_label = 'Not Available'
            else:
                if t.true_label in predictions_value:
                    labelidx = predictions_value.index(t.true_label)

                    if predictions_proba[labelidx] <threshold:
                        prediction_label = 'Anomalous'
                else:
                    prediction_label = 'Anomalous'

            if t.true_label != 'End':
                prediction_list.append(prediction_label)


        true_label_list = []

        labellist = list(df['noise'])
        actlist = list(df['Activity'])
        for pos, t in enumerate(labellist):
            if t == 'Start' or t == 'End':
                continue
            elif t == 'true':
                true_label = 'Anomalous'
            else:
                true_label = 'Normal'
            true_label_list.append(true_label)


        for pos, p in enumerate(prediction_list):
            global_pred.append(p)
            global_true.append(true_label_list[pos])


    saving_data = {'y_true':global_true, 'y_pred':global_pred}
    import pickle
    saving_file_name = file_name.split('/')[-1][:-4]

    with open('./result/xgb_thr%s_window%s_%s.pkl'%(threshold, window_size, saving_file_name), 'wb') as fp:
        pickle.dump(saving_data, fp)
